In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Nomenclature
- Complete data collection procedure, from init to writing is referred to as a "run".   
- Each step in a run is an "event".  
- Where a run is in the sequence of events is it's "state"
- To move between states requires "triggers"

## States are
* waiting: in between events; triggered by conclusion of steps.  Options while waiting include
    * set parameters (wavelength/steps, integration time, devices to include in the initialization, compression?, write results?, display in real time?)
    * initialize
    * run all
* initializing (complete and between runs): triggered by initialize command (at any time, but warning if initialize already happened) will initialize all devices with checked boxes in GUI. Returns dictionary of device metadata
* collecting: triggered by collection command (at end sets self.data_in_queue = True)
* compressing: triggered by compression commmand (at end sets self.data_ready_to_store = True) 
* writing: triggered by writing command
* fubar: triggered by an error in any of the steps
* interupt: is this a state or a command?  

In [2]:
import sys
#sys.path.append(r'C:\Users\marco viero\Repositories\SPHEREx-lab-tools\SPHEREx-Calibration-Automation\pylab\pylabcal\pylabcald')
sys.path.append(r'..\pylab\pylabcal\pylabcald')

from pylabcaldlib.instruments.repeatedtimer import RepeatedTimer
from pylabcaldlib.instruments.powermaxusb import PowermaxUSB

In [3]:
import pdb
import numpy as np
class SM:
    
    startState = 'waiting'
    init = False
    in_queue = False
    ready_to_store = False
    
    error_status = False
    
    def start(self):
        self.state = self.startState
        self.init_status = self.init
        self.data_in_queue = self.in_queue
        self.data_ready_to_store = self.ready_to_store
        self.errorStatus = self.error_status
        self.errorDict = {}
        self.metadata = {}
        
    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s

        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

In [4]:
import sys
sys.path.append(r'C:\Users\marco viero\Repositories\SPHEREx-lab-tools\SPHEREx-Calibration-Automation\pylab\pylabcal\pylabcald')

from pylabcaldlib.instruments.repeatedtimer import RepeatedTimer
from pylabcaldlib.instruments.powermaxusb import PowermaxUSB
from pylabcaldlib.instruments.serial_motor_dpy50601 import DPY50601

class SpectralCalibrationMachine(SM):
    
    def __init__(self):
        print('Start by Initializing')
               
        #self.errorStatus = self.initialize()
        self.initialize()
        #pdb.set_trace()

    def initialize(self):
        print('Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict')
        
        self.start()
        
        # Load Thorlabs Laser
        
        # Load Filter Wheels
        
        # Load Data Collection
        
        # Load Powermax
        self.powermax = PowermaxUSB()
        #self.errorDict['powermax'] = self.powermax.error_record 
        
        # Load First Motor Controller
        self.xstage = DPY50601(0)
        
        #pdb.set_trace()
        if np.sum(sum(self.errorDict.values())):
            #self.error_handler()
            self.errorStatus = True
            #return 
        else:
            self.init_status = True
            
        return 'waiting', 'Initialized without errors' #errors 
    
    def error_handler(self):
        for key, value in self.errorDict.items():
            #pdb.set_trace()
            print(key,':',value)
            #self.errorStatus = True
        
        self.errorStatus = False
        return 'waiting', 'Debug Errors' #errors 
    
    def collect_data(self):
        print('Collecting Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_in_queue = True
        return 'waiting', 'Data Collected' #errors 
    
    def compress_data(self):
        print('Compressing Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
        return 'waiting', 'Data Compressed' #errors 
    
    def write_data_to_disk(self):
        print('Writing Data to Disk')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
        return 'waiting', 'Data Written to Disk' #errors 
    
    def generateOutput(self,state):
        if state == 'initializing':
            return self.initialize()
        elif state == 'collecting':
            return self.collect_data()
        elif state == 'compressing':
            return self.compress_data()
        elif state == 'storing':
            return self.write_data_to_disk()
        elif state == 'error':
            return self.error_handler()        
        else:
            return 'wait'
     
    def getNextValues(self, state, dict_in):
        #pdb.set_trace()
        if self.errorStatus == True:
            nextState = 'error'
        elif state == 'waiting' and 'initialize' in dict_in:
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and 'collect' in dict_in:
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and 'compress' in dict_in:
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and 'write_to_disk' in dict_in:
            nextState = 'storing'
        else:
            nextState = 'error'
        return (self.generateOutput(nextState))

In [5]:
scm2 = SpectralCalibrationMachine()
print(scm2.powermax.error_record)

Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
1


In [6]:
scm2.transduce([{'initialize': [0,2,4]}])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...


['Initialized without errors']

In [7]:
scm2.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Compressing Data
Writing Data to Disk


['Initialized without errors',
 'Data Collected',
 'Data Compressed',
 'Data Written to Disk']

In [8]:
scm2.transduce(['initialize','collect','compress','write_to_disk'])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Compressing Data
Writing Data to Disk


['Initialized without errors',
 'Data Collected',
 'Data Compressed',
 'Data Written to Disk']

# GUI Developement

In [9]:
import sys
from math import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

In [10]:
class MainWindow(QMainWindow):

    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)
        
        self.setWindowTitle("My Awesome App")
        
        layout = QGridLayout()
        
        self.initialize_button = QPushButton(self)
        self.initialize_button.setText("Initialize")
        layout.addWidget(self.initialize_button, 0, 0)
        
        self.setup_button = QPushButton(self)
        self.setup_button.setText("Setup")
        layout.addWidget(self.setup_button, 1, 0)

        self.automate_box = QCheckBox(self)
        self.automate_box.setText("Automate")
        layout.addWidget(self.automate_box, 1, 1)

        self.start_run_button = QPushButton(self)
        self.start_run_button.setText("Start Run")
        layout.addWidget(self.start_run_button, 0, 1)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        self.retranslateUi()

    def retranslateUi(self):
        
        self.initialize_button.clicked.connect(self.initialize_button_clicked)
        
        self.setup_button.clicked.connect(self.setup_button_clicked)
        
        self.start_run_button.clicked.connect(self.start_run_button_clicked)
        #self.button2.clicked.connect(self.initialize_button_clicked)
        
        self.dialogTextBrowser = setupWindow(self)

    def initialize_button_clicked(self):
        
        self.scm2 = SpectralCalibrationMachine()

        
    @pyqtSlot()
    def setup_button_clicked(self):
        self.dialogTextBrowser.exec_()
        

        
    def start_run_button_clicked(self):
        
        scm2.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

    def clicked(self):
        button = self.sender()
        if button is None or not isinstance(button, QPushButton):
            return
        self.label.setText("You clicked button '{}'".format(
                           button.text()))
        
class setupWindow(QDialog):
    def __init__(self, parent=None):
        super(setupWindow, self).__init__(parent)

        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.textBrowser = QTextBrowser(self)
        self.textBrowser.append("This is the Setup Window!")

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.textBrowser)
        self.verticalLayout.addWidget(self.buttonBox)
        
        self.metadata_button = QPushButton(self)
        self.metadata_button.setText("Metadata")
        self.metadata_button.clicked.connect(self.metadata_button_clicked)
        self.dialogTextBrowser = metadataWindow(self)

    @pyqtSlot()
    def metadata_button_clicked(self):
        self.dialogTextBrowser.exec_()
        
class metadataWindow(QDialog):
    def __init__(self, parent=None):
        super(metadataWindow, self).__init__(parent)

        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.textBrowser = QTextBrowser(self)
        self.textBrowser.append("This is the Metadata window!")

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.textBrowser)
        self.verticalLayout.addWidget(self.buttonBox)

In [11]:
class Form(QDialog):

    def __init__(self, parent=None):
        super(Form, self).__init__(parent)
        
        self.initialize_button = QPushButton("Initialize")
        
        self.setup_button = QPushButton(self)
        self.setup_button.setText("Setup")
        
        self.automate_box = QCheckBox(self)
        self.automate_box.setText("Automate")

        self.start_run_button = QPushButton("Start Run")

        
        
        self.label = QLabel("Click a button...")

        layout = QHBoxLayout()
        layout.addWidget(self.initialize_button)
        
        
        layout.addWidget(self.setup_button)
        layout.addWidget(self.automate_box)
        
        
        layout.addWidget(self.start_run_button)
        layout.addStretch()
        layout.addWidget(self.label)
        self.setLayout(layout)
        
        #self.buttoncallback = lambda who="Button": self.anyButton(who)
        
        #self.button1.connect(SIGNAL("clicked()"), self.one)
        #self.button2callback = functools.partial(self.anyButton, "Two")
        #self.connect(button2, SIGNAL("clicked()"),
        #             self.button2callback)
        #self.button3callback = lambda who="Three": self.anyButton(who)
        #self.connect(button3, SIGNAL("clicked()"),
        #             self.button3callback)
        #self.connect(button4, SIGNAL("clicked()"), self.clicked)
        #self.connect(button5, SIGNAL("clicked()"), self.clicked)

        #self.setWindowTitle("Connections")

        self.retranslateUi()

    def retranslateUi(self):
        
        self.initialize_button.clicked.connect(self.initialize_button_clicked)
        
        self.setup_button.clicked.connect(self.setup_button_clicked)
        
        self.start_run_button.clicked.connect(self.start_run_button_clicked)
        #self.button2.clicked.connect(self.initialize_button_clicked)
        
        self.dialogTextBrowser = setupWindow(self)

    def initialize_button_clicked(self):
        
        self.scm2 = SpectralCalibrationMachine()

        
    @pyqtSlot()
    def setup_button_clicked(self):
        self.dialogTextBrowser.exec_()
        

        
    def start_run_button_clicked(self):
        
        scm2.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

    def clicked(self):
        button = self.sender()
        if button is None or not isinstance(button, QPushButton):
            return
        self.label.setText("You clicked button '{}'".format(
                           button.text()))
        
class setupWindow(QDialog):
    def __init__(self, parent=None):
        super(setupWindow, self).__init__(parent)

        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.textBrowser = QTextBrowser(self)
        self.textBrowser.append("This is the Setup Window!")

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.textBrowser)
        self.verticalLayout.addWidget(self.buttonBox)
        
        self.metadata_button = QPushButton(self)
        self.metadata_button.setText("Metadata")
        self.metadata_button.clicked.connect(self.metadata_button_clicked)
        self.dialogTextBrowser = metadataWindow(self)

    @pyqtSlot()
    def metadata_button_clicked(self):
        self.dialogTextBrowser.exec_()
        
class metadataWindow(QDialog):
    def __init__(self, parent=None):
        super(metadataWindow, self).__init__(parent)

        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)

        self.textBrowser = QTextBrowser(self)
        self.textBrowser.append("This is the Metadata window!")

        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.textBrowser)
        self.verticalLayout.addWidget(self.buttonBox)

In [ ]:
app = QApplication(sys.argv)
#form = Form()
form = MainWindow()
form.show()
app.exec_()

In [ ]:
class SM:
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

class Accumulator(SM):
    #startState = 0
    
    def __init__(self, initialValue):
        self.startState = initialValue

    def getNextValues(self, state, inp):
        return state + inp, state + inp

In [ ]:
a = Accumulator()
a.start()

In [ ]:
a = Accumulator(2)
a.start()

In [ ]:
a.step(3)

In [ ]:
a = Accumulator(5)
a.transduce([5,3])

In [ ]:
class SM:
    
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

class Accumulator(SM):
    #startState = 0
    
    def __init__(self, initialValue):
        self.startState = initialValue

    def getNextValues(self, state, inp):
        return state + inp, state + inp

In [ ]:
class SM:
    
    initialized = False
    errorStatus = 0
    
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

In [ ]:
class SpectralCalibrationMachine(SM):
    
    startState = 'waiting'
    
    def __init__(self):
        print('Launch GUI')
        print('Start by Initialization')
        self.initialize()
        
    def initialize(self):
        print('Run Initialization Procedure')
        self.intialized = True
            
    def generateOutput(self,state):
        if state == 'initializing':
            return 'initialize'
        elif state == 'collecting':
            return 'collect'
        elif state == 'compressing':
            return 'compress'
        elif state == 'storing':
            return 'write to disk'
        else:
            return 'wait' 
        
    # Inputs to consider: initStatus, errorStatus, rawDataCompressReady, rawDataCompressed, dataWriteReady, dataWritten
    def getNextValues(self, state, inp):
        #(initStatus, errorStatus, rawDataCompressReady, rawDataCompressed, dataWriteReady, dataWritten) = inp
        #pdb.set_trace()
        if self.errorStatus == True:
            nextState = 'error' 
        elif state == 'waiting' and inp == 'initialize':
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and inp == 'collect':
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and inp == 'compress':
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and inp == 'write_to_disk':
            nextState = 'storing'
        else:
            nextState = 'error'
            
        return (nextState, self.generateOutput(nextState))

In [ ]:
scm = SpectralCalibrationMachine()

In [ ]:
scm.start()

In [ ]:
inp = (False, False, False, False, False, False)
scm.transduce([inp])

In [ ]:
scm.state